In [1]:
import re
import json
import csv
import requests
from bs4 import BeautifulSoup ,SoupStrainer
import pandas as pd 
from io import StringIO
from pathlib import Path  

!pip install selenium

import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

import sys

import unittest

import time


In [33]:
def get_infinity_page_html_data(chromedriver_path="./chromedriver.exe", scroll_number=50, sleep_time=2, url = ""):
    # Create selenium driver
    s = Service(chromedriver_path)
    driver = webdriver.Chrome(service=s)
    driver.get(url)

    for i in range(1,scroll_number):
        print(f"scroll_number: {i}")
        driver.execute_script("window.scrollTo(1,100000)")
        time.sleep(sleep_time)
    html_data = driver.page_source
    driver.close()
    
    soup = BeautifulSoup(html_data, 'html.parser')    
    return soup


In [34]:
def make_table(soup):
    table = soup.find_all('table')
    df = pd.read_html(str(table))[0]
    return df

In [35]:
def data_cleaning(df):
    df = df[df["Open"].str.contains("Dividend") == False]
    df = df[df["Open"].str.contains("Stock") == False]
    df.drop(df.tail(1).index,inplace=True)
    return df

In [36]:
def write_to_file(df, path):
    from pathlib import Path
    filepath = Path(path)
    filepath.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(filepath) 

In [39]:
base_url = "https://finance.yahoo.com/quote/"
period_url = "/history?period1=1262304000&period2=1651881600&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true"
indexes = ["NVDA","AAPL","TSLA","AMD","MSFT","INTC"]
main_df = pd.DataFrame()

for index in indexes:
    soup = get_infinity_page_html_data(url = (base_url + index + period_url))
    table = soup.find_all('table')
    df = pd.read_html(str(table))[0]
    
    df = data_cleaning(df)
    
    path = index + ".csv"
    write_to_file(df, path)
    
    print()
    
   
    main_df['Date'] = df['Date']
    main_df[index + " " + "Open"] = df['Open']
    main_df[index + " " + "High"] = df['High']
    main_df[index + " " + "Low"] = df['Low']
    main_df[index + " " + "Close"] = df['Close*']
    main_df[index + " " + "Volume"] = df['Volume']
    main_df[index + " " +'Daily change'] = (((df['Close*']).astype(float))-((df['Open']).astype(float)))/((df['Open']).astype(float))*100
    main_df[index + " " +'Daily change Max'] =  abs((df['High'].astype(float)-df['Low'].astype(float))/df['Low'].astype(float)*100)
    


    
    print(df)
    print(df.info())

    
write_to_file(main_df, ("main_dataframe" + ".csv"))
main_df

scroll_number: 1
scroll_number: 2
scroll_number: 3
scroll_number: 4
scroll_number: 5
scroll_number: 6
scroll_number: 7
scroll_number: 8
scroll_number: 9
scroll_number: 10
scroll_number: 11
scroll_number: 12
scroll_number: 13
scroll_number: 14
scroll_number: 15
scroll_number: 16
scroll_number: 17
scroll_number: 18
scroll_number: 19
scroll_number: 20
scroll_number: 21
scroll_number: 22
scroll_number: 23
scroll_number: 24
scroll_number: 25
scroll_number: 26
scroll_number: 27
scroll_number: 28
scroll_number: 29
scroll_number: 30
scroll_number: 31
scroll_number: 32
scroll_number: 33
scroll_number: 34
scroll_number: 35
scroll_number: 36
scroll_number: 37
scroll_number: 38
scroll_number: 39
scroll_number: 40
scroll_number: 41
scroll_number: 42
scroll_number: 43
scroll_number: 44
scroll_number: 45
scroll_number: 46
scroll_number: 47
scroll_number: 48
scroll_number: 49

              Date    Open    High     Low  Close* Adj Close**    Volume
0     May 06, 2022  187.36  195.12  179.90  186.75   

scroll_number: 1
scroll_number: 2
scroll_number: 3
scroll_number: 4
scroll_number: 5
scroll_number: 6
scroll_number: 7
scroll_number: 8
scroll_number: 9
scroll_number: 10
scroll_number: 11
scroll_number: 12
scroll_number: 13
scroll_number: 14
scroll_number: 15
scroll_number: 16
scroll_number: 17
scroll_number: 18
scroll_number: 19
scroll_number: 20
scroll_number: 21
scroll_number: 22
scroll_number: 23
scroll_number: 24
scroll_number: 25
scroll_number: 26
scroll_number: 27
scroll_number: 28
scroll_number: 29
scroll_number: 30
scroll_number: 31
scroll_number: 32
scroll_number: 33
scroll_number: 34
scroll_number: 35
scroll_number: 36
scroll_number: 37
scroll_number: 38
scroll_number: 39
scroll_number: 40
scroll_number: 41
scroll_number: 42
scroll_number: 43
scroll_number: 44
scroll_number: 45
scroll_number: 46
scroll_number: 47
scroll_number: 48
scroll_number: 49

              Date    Open    High     Low  Close* Adj Close**    Volume
0     May 06, 2022  274.81  279.25  271.27  274.73   

,Date,NVDA Open,NVDA High,NVDA Low,NVDA Close,NVDA Volume,NVDA Daily change,NVDA Daily change Max,AAPL Open,AAPL High,...,MSFT Volume,MSFT Daily change,MSFT Daily change Max,INTC Open,INTC High,INTC Low,INTC Close,INTC Volume,INTC Daily change,INTC Daily change Max
0,"May 06, 2022",187.36,195.12,179.90,186.75,63299300,-0.325576,8.460256,156.01,159.44,...,37748300,-0.029111,2.941719,44.49,45.15,43.84,44.30,40910000,-0.427062,2.988139
1,"May 05, 2022",198.67,199.25,185.00,188.44,62633100,-5.149242,7.702703,NaN,NaN,...,43260400,-2.868250,4.377779,45.60,45.91,44.13,44.60,52996700,-2.192982,4.033537
2,NaN,199.23,204.00,187.51,203.34,64885500,2.062942,8.794198,163.85,164.08,...,33599300,2.615096,5.113287,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"May 04, 2022",194.00,198.25,191.33,196.02,47575100,1.041237,3.616788,159.67,166.48,...,25978600,-0.767714,1.420667,45.06,46.64,44.41,46.54,44625600,3.284510,5.021392
4,"May 03, 2022",185.41,195.74,183.91,195.33,57204900,5.350305,6.432494,158.15,160.71,...,35151100,2.434194,3.156904,44.79,45.49,44.23,45.06,40158400,0.602813,2.848745
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2695,"Oct 21, 2011",3.88,3.88,3.71,3.73,61178000,-3.865979,4.582210,14.29,14.30,...,76300200,-0.807043,3.560606,23.86,24.25,23.80,24.03,123273100,0.712490,1.890756
2696,"Oct 20, 2011",3.93,3.97,3.84,3.93,70245200,0.000000,3.385417,14.33,14.59,...,42880000,-0.876872,1.703073,23.88,23.88,23.35,23.61,88682000,-1.130653,2.269807
2697,"Oct 19, 2011",3.63,3.87,3.63,3.87,85483200,6.611570,6.611570,15.06,15.17,...,52487900,1.373422,2.238806,24.01,24.50,23.86,24.24,139956200,0.957934,2.682313
2698,"Oct 18, 2011",3.72,3.73,3.65,3.65,61206400,-1.881720,2.191781,15.06,15.24,...,39453300,-0.479528,2.122905,23.17,23.49,23.07,23.40,85272200,0.992663,1.820546


In [40]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2661 entries, 0 to 2699
Data columns (total 43 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Date                   2618 non-null   object 
 1   NVDA Open              2661 non-null   object 
 2   NVDA High              2661 non-null   object 
 3   NVDA Low               2661 non-null   object 
 4   NVDA Close             2661 non-null   object 
 5   NVDA Volume            2661 non-null   object 
 6   NVDA Daily change      2661 non-null   float64
 7   NVDA Daily change Max  2661 non-null   float64
 8   AAPL Open              2619 non-null   object 
 9   AAPL High              2619 non-null   object 
 10  AAPL Low               2619 non-null   object 
 11  AAPL Close             2619 non-null   object 
 12  AAPL Volume            2619 non-null   object 
 13  AAPL Daily change      2619 non-null   float64
 14  AAPL Daily change Max  2619 non-null   float64
 15  TSLA